In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="thG33xUjGSCIzbllIF4X")
project = rf.workspace("seher-pvagg").project("yatak-yarasi")
version = project.version(1)
dataset = version.download("multiclass")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to yatak-yarasi-1 in multiclass:: 100%|██████████| 1796/1796 [00:01<00:00, 1519.72it/s]


In [ ]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [ ]:
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp.solutions.pose.Pose(
                              model_complexity=2,
                              min_detection_confidence=0.5,
                              min_tracking_confidence=0.5
                              )

In [ ]:
# Path to the dataset directory and DataFrame
train_dir = '/content/yatak-yarasi-1/train'
df = pd.read_csv('/content/yatak-yarasi-1/train/_classes.csv')  # Update path if necessary
df.columns = df.columns.str.strip()
df['labels'] = df[['saga yatik', 'sirt ustu', 'sola yatik', 'yuz ustu']].apply(
    lambda row: [label for label, value in zip(['saga yatik', 'sirt ustu', 'sola yatik', 'yuz ustu'], row) if value == 1],
    axis=1)

# List to store pose estimations for each image
pose_estimations = []

# Iterate through each image in the dataset
for index, row in df.iterrows():
    image_path = os.path.join(train_dir, row['filename'])
    image = cv2.imread(image_path)

    if image is None:
        print(f"Warning: Image {row['filename']} not found in {train_dir}")
        continue

    # Convert the image to RGB for Mediapipe processing
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    rows, cols, _ = image.shape
    landmarks = []

    # Process pose landmarks if detected
    if results.pose_landmarks:
        if len(results.pose_landmarks.landmark) == 33:
            # Loop through each of the 33 pose landmarks and store their y coordinates
            for lm in results.pose_landmarks.landmark:
                cy = int(lm.y * rows)  # Get y-coordinate (scaled to image height)
                landmarks.append(cy)  # Append the y-coordinate

            # Append the pose estimation and corresponding class label to the list
            pose_estimations.append({"class_label": row['labels'], "pose": landmarks})

# Convert the pose estimations to a DataFrame for saving or further processing
pose_df = pd.DataFrame(pose_estimations)

# Flatten the 'pose' column (which contains lists) into separate columns for each point
pose_df_expanded = pose_df['pose'].apply(pd.Series)

# Explicitly assign column names for 33 pose points
pose_df_expanded.columns = [f"point_{i+1}" for i in range(33)]

# Combine the class label with the pose points
pose_df_expanded['class_label'] = pose_df['class_label']

# Reorder columns to have class label first
pose_df_expanded = pose_df_expanded[['class_label'] + [f"point_{i+1}" for i in range(33)]]

# Save to CSV
pose_df_expanded.to_csv('pose_estimations_y_only_train.csv', index=False)
print("Pose estimation (y-coordinates only) with class labels saved to pose_estimations_y_only.csv")


Pose estimation (y-coordinates only) with class labels saved to pose_estimations_y_only.csv


In [ ]:
df1=pd.read_csv("/content/pose_estimations_y_only_train.csv")
df1.head()

,class_label,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,...,point_24,point_25,point_26,point_27,point_28,point_29,point_30,point_31,point_32,point_33
0,['saga yatik'],323,330,333,336,327,328,329,338,345,...,211,212,312,306,333,317,323,312,357,361
1,['sirt ustu'],359,368,368,367,370,372,373,368,378,...,218,218,321,343,406,387,404,374,459,437
2,['yuz ustu'],428,437,440,444,427,425,425,447,419,...,273,262,200,200,92,92,73,79,53,53
3,['yuz ustu'],449,449,450,450,447,446,445,446,435,...,261,260,194,195,111,110,88,81,105,105
4,['sola yatik'],441,456,458,460,454,455,461,462,465,...,204,183,91,58,59,7,53,-2,23,-32


In [ ]:
df1.shape #779train

(786, 34)

In [ ]:
# Path to the dataset directory and DataFrame
valid_dir = '/content/yatak-yarasi-1/valid'
df = pd.read_csv('/content/yatak-yarasi-1/valid/_classes.csv')  # Update path if necessary
df.columns = df.columns.str.strip()
df['labels'] = df[['saga yatik', 'sirt ustu', 'sola yatik', 'yuz ustu']].apply(
    lambda row: [label for label, value in zip(['saga yatik', 'sirt ustu', 'sola yatik', 'yuz ustu'], row) if value == 1],
    axis=1)

# List to store pose estimations for each image
pose_estimations = []

# Iterate through each image in the dataset
for index, row in df.iterrows():
    image_path = os.path.join(valid_dir, row['filename'])
    image = cv2.imread(image_path)

    if image is None:
        print(f"Warning: Image {row['filename']} not found in {train_dir}")
        continue

    # Convert the image to RGB for Mediapipe processing
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    rows, cols, _ = image.shape
    landmarks = []

    # Process pose landmarks if detected
    if results.pose_landmarks:
        if len(results.pose_landmarks.landmark) == 33:
            # Loop through each of the 33 pose landmarks and store their y coordinates
            for lm in results.pose_landmarks.landmark:
                cy = int(lm.y * rows)  # Get y-coordinate (scaled to image height)
                landmarks.append(cy)  # Append the y-coordinate

            # Append the pose estimation and corresponding class label to the list
            pose_estimations.append({"class_label": row['labels'], "pose": landmarks})

# Convert the pose estimations to a DataFrame for saving or further processing
pose_df = pd.DataFrame(pose_estimations)

# Flatten the 'pose' column (which contains lists) into separate columns for each point
pose_df_expanded = pose_df['pose'].apply(pd.Series)

# Explicitly assign column names for 33 pose points
pose_df_expanded.columns = [f"point_{i+1}" for i in range(33)]

# Combine the class label with the pose points
pose_df_expanded['class_label'] = pose_df['class_label']

# Reorder columns to have class label first
pose_df_expanded = pose_df_expanded[['class_label'] + [f"point_{i+1}" for i in range(33)]]

# Save to CSV
pose_df_expanded.to_csv('pose_estimations_y_only_valid.csv', index=False)
print("Pose estimation (y-coordinates only) with class labels saved to pose_estimations_y_only.csv")

Pose estimation (y-coordinates only) with class labels saved to pose_estimations_y_only.csv


In [ ]:
df2=pd.read_csv("/content/pose_estimations_y_only_valid.csv")
df2.head()

,class_label,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,...,point_24,point_25,point_26,point_27,point_28,point_29,point_30,point_31,point_32,point_33
0,['sola yatik'],295,294,293,293,293,293,293,287,284,...,372,334,387,228,454,156,466,129,466,104
1,['sirt ustu'],277,272,275,278,263,260,257,281,254,...,424,399,486,450,545,519,555,534,559,534
2,['sirt ustu'],156,141,139,138,142,142,142,136,139,...,351,328,458,410,539,464,552,480,589,508
3,['sola yatik'],337,340,343,345,332,328,322,343,303,...,412,391,449,404,413,384,402,388,412,364
4,['sirt ustu'],283,281,281,281,281,281,282,285,286,...,377,389,436,449,505,531,521,540,508,558


In [ ]:
df2.shape #87valid

(62, 34)

In [ ]:
# Path to the dataset directory and DataFrame
test_dir = '/content/yatak-yarasi-1/test'
df = pd.read_csv('/content/yatak-yarasi-1/test/_classes.csv')  # Update path if necessary
df.columns = df.columns.str.strip()
df['labels'] = df[['saga yatik', 'sirt ustu', 'sola yatik', 'yuz ustu']].apply(
    lambda row: [label for label, value in zip(['saga yatik', 'sirt ustu', 'sola yatik', 'yuz ustu'], row) if value == 1],
    axis=1)

# List to store pose estimations for each image
pose_estimations = []

# Iterate through each image in the dataset
for index, row in df.iterrows():
    image_path = os.path.join(test_dir, row['filename'])
    image = cv2.imread(image_path)

    if image is None:
        print(f"Warning: Image {row['filename']} not found in {train_dir}")
        continue

    # Convert the image to RGB for Mediapipe processing
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    rows, cols, _ = image.shape
    landmarks = []

    # Process pose landmarks if detected
    if results.pose_landmarks:
        if len(results.pose_landmarks.landmark) == 33:
            # Loop through each of the 33 pose landmarks and store their y coordinates
            for lm in results.pose_landmarks.landmark:
                cy = int(lm.y * rows)  # Get y-coordinate (scaled to image height)
                landmarks.append(cy)  # Append the y-coordinate

            # Append the pose estimation and corresponding class label to the list
            pose_estimations.append({"class_label": row['labels'], "pose": landmarks})

# Convert the pose estimations to a DataFrame for saving or further processing
pose_df = pd.DataFrame(pose_estimations)

# Flatten the 'pose' column (which contains lists) into separate columns for each point
pose_df_expanded = pose_df['pose'].apply(pd.Series)

# Explicitly assign column names for 33 pose points
pose_df_expanded.columns = [f"point_{i+1}" for i in range(33)]

# Combine the class label with the pose points
pose_df_expanded['class_label'] = pose_df['class_label']

# Reorder columns to have class label first
pose_df_expanded = pose_df_expanded[['class_label'] + [f"point_{i+1}" for i in range(33)]]

# Save to CSV
pose_df_expanded.to_csv('pose_estimations_y_only_test.csv', index=False)
print("Pose estimation (y-coordinates only) with class labels saved to pose_estimations_y_only.csv")

Pose estimation (y-coordinates only) with class labels saved to pose_estimations_y_only.csv


In [ ]:
df=pd.read_csv("/content/pose_estimations_y_only_test.csv")
df.head()

,class_label,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,...,point_24,point_25,point_26,point_27,point_28,point_29,point_30,point_31,point_32,point_33
0,['sirt ustu'],104,101,103,106,98,98,97,116,105,...,291,278,386,357,488,466,500,487,526,498
1,['yuz ustu'],129,115,115,115,115,114,114,125,120,...,351,338,391,385,544,478,566,495,568,498
2,['yuz ustu'],326,322,322,321,323,323,323,316,318,...,330,327,411,381,448,350,452,348,457,341
3,['sola yatik'],497,506,506,505,509,510,512,504,512,...,320,329,249,254,153,148,146,141,106,101
4,['sola yatik'],360,367,370,372,364,364,363,387,376,...,307,303,265,297,364,331,369,334,404,352


In [ ]:
df.shape #38test

(37, 34)

In [ ]:
data=pd.concat([df1,df2,df],axis=0)
data.shape

(885, 34)

In [ ]:
data.duplicated().sum()

0

In [ ]:
data.to_csv("pose_estimation_data.csv",index=False)